In [36]:
### COMPLEX NUMERICAL SIMULATOR ###
import numpy as np
import torch
import scipy.integrate
def numerical_schrodinger_cmplx(psi0, v, ts, length=1, size=100, verbose=False):
    xs = np.linspace(0,length,size)
    dx = length/size
    
    # construct laplacian operator and then Hamiltonian
    D2 = -2*np.eye(size)
    for i in range(size-1):
        D2[i,i+1] = 1
        D2[i+1,i] = 1
    H = -0.5*D2/(dx**2) + np.diag(v(xs))
    
    y0 = psi0(xs)*(1+0j)
    
    if verbose:
        energy = y0.transpose()@H@y0/(y0.transpose()@y0)
        print(f'Energy expectation value is {energy}.')
    
    sol = scipy.integrate.solve_ivp(lambda t,p: H@p / 1j, t_span=[0,np.max(ts)], y0=y0, t_eval=ts, method="RK23")
    return sol.t, sol.y

In [55]:
### REAL NUMERICAL SIMULATOR ###
import scipy.integrate
def numerical_schrodinger_real(psi0, v, ts, length=1, size=100, verbose=False):
    xs = np.linspace(0,length,size)
    dx = length/size
    
    # construct laplacian operator and then Hamiltonian
    D2 = -2*np.eye(size)
    for i in range(size-1):
        D2[i,i+1] = 1
        D2[i+1,i] = 1
    
    off_diags = np.array([[0,1],[-1,0]])
    
    H = np.kron(off_diags, -0.5*D2/(dx**2) + np.diag(v(xs)))
    
    y0_cmplx = psi0(xs)*(1+0j)
    y0_real = np.concatenate((y0_cmplx.real, y0_cmplx.imag))
    
    #sol = scipy.integrate.solve_ivp(lambda t,p: H@p, t_span=[0,np.max(ts)], y0=y0_real, t_eval=ts, method="RK23")
    sol = scipy.integrate.odeint(lambda p,t: H@p, t=ts, y0=y0_real)
    
    sol_cmplx = sol[:,:size] + 1j*sol[:,size:]
    
    return ts, sol_cmplx

In [54]:
from timeit import timeit

cplx = timeit(lambda: numerical_schrodinger_cmplx(lambda x: np.sin(np.pi*x), lambda x: 0*x, np.linspace(0,10,500), size=25), number=5)
real = timeit(lambda: numerical_schrodinger_real(lambda x: np.sin(np.pi*x), lambda x: 0*x, np.linspace(0,10,500), size=25), number=5)

print(f'Complex takes {cplx*1000} milliseconds.')
print(f'Real takes {real*1000} milliseconds.')

(500,)
(500, 50)
(500,)
(500, 50)
(500,)
(500, 50)
(500,)
(500, 50)
(500,)
(500, 50)
Complex takes 2440.06416800039 milliseconds.
Real takes 1184.8156490013935 milliseconds.
